In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os 

In [2]:
### Remove the unncessary characters
text_file = open('text file.txt', 'r', encoding='utf8')
lines = text_file.read()
data_into_list = data = lines.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')
data_into_list

# Remove the unnecessary spaces
data = data_into_list.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of Pride and Prejudice, by Jane AustenThis eBook is for the use of anyone anywhere in the United States andmost other parts of the world at no cost and with almost no restrictionswhatsoever. You may copy it, give it away or re-use it under the termsof the Project Gutenberg License included with this eBook or online atwww.gutenberg.org. If you are not located in the United States, youwill have to check the laws of the country where you are located beforeusing this eBoo'

##### Tokenization

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

### saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 223, 186, 912, 3, 328, 4, 1351, 30, 72, 4174, 912, 23, 21, 1]

In [4]:
len(sequence_data)

125076

In [5]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

7200


##### We will use three words to preidct the next word

In [6]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print(f'The length of sequneces are : {len(sequences)}')
sequences[:10]

The length of sequneces are : 125073


[[1, 223, 186, 912],
 [223, 186, 912, 3],
 [186, 912, 3, 328],
 [912, 3, 328, 4],
 [3, 328, 4, 1351],
 [328, 4, 1351, 30],
 [4, 1351, 30, 72],
 [1351, 30, 72, 4174],
 [30, 72, 4174, 912],
 [72, 4174, 912, 23]]

##### Separate input and output

In [7]:
x = []
y = []

for i in sequences:
    x.append(i[0:3])
    y.append(i[3])

x = np.array(x)
y = np.array(y)


### Label the output categorically

In [8]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

##### Defining the model

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             72000     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7200)              7207200   
                                                                 
Total params: 20,328,200
Trainable params: 20,328,200
Non-trainable params: 0
_________________________________________________________________


##### Training the model

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))
model.fit(x, y, epochs=70, batch_size=64, callbacks=[checkpoint])

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.